In [344]:
import numpy as np
from numpy import sin, cos, multiply, transpose, cross, dot
from sympy.physics.vector import dynamicsymbols
from sympy import *
init_printing(pretty_print=True)

# Symbol Declaration
## Theta
t = Symbol('t')
t_1 = Function('theta_1')(t)
t_2 = Function('theta_2')(t)
d_3 = Function('d_3')(t)
t_4 = Function('theta_4')(t)

## Constant
r_1, r_2, r_3, r_4 = symbols('r_1 r_2 r_3 r_4')
d_1, d_2 = symbols('d_1 d_2')
a_1, a_2, a_3 = symbols('a_1 a_2 a_3')
I_xx1, I_yy1, I_zz1 = symbols('I_xx1 I_yy1 I_zz1')
I_xx2, I_yy2, I_zz2 = symbols('I_xx2 I_yy2 I_zz2')
I_xx3, I_yy3, I_zz3 = symbols('I_xx3 I_yy3 I_zz3')
m_1, m_2, m_3, m_4 = symbols('m_1, m_2, m_3, m_4')

## Gravity
g = symbols('g')


## Special Case

In [345]:
z_q = (np.array([0, 0, 1])).reshape((3, 1))
x_q = (np.array([1, 0, 0])).reshape((3, 1))
o_q = (np.array([0, 0, 0])).reshape((3, 1))

## P and Pc Parameter

### P

In [346]:
P_01 = np.array([0 ,0, d_1]).reshape(3, -1)
P_12 = np.array([a_1,0, d_2]).reshape(3, -1)
P_23 = np.array([a_2, 0, d_3]).reshape(3, -1)
P_34 = o_q

### Pc

In [347]:
P_1c = r_1*x_q
P_2c = r_2*x_q
P_3c = r_3*z_q
P_4c = o_q

## Joint Handler

In [348]:
def R(theta):  # R n+1 -> n
    return np.array([
        [cos(theta), -sin(theta), 0],
        [sin(theta), cos(theta),  0],
        [0,          0,           1]
    ])

In [349]:
class jointHandler():
    
    def __init__(self,_P, _Pc, _m, _r, _t, _w, I_, _d, _v, _ROT, _Pot):
        self.P = _P
        self.P_c = _Pc
        self.m = _m
        self.R = R(_t)
        self.r = _r
        self.w = _w
        self.td = diff(_t)
        self.I = I_
        self.d_dot = diff(_d)
        self.v = _v
        self.ROT = _ROT
        self.Pot = _Pot - _P[2]+ _Pc[2]
    
    def getI(self):
        I = self.I
        return np.array([
                [I[0], 0, 0],
                [0, I[1], 0],
                [0, 0,  I[2]]
                ])
    
    def getRotation_w(self):                          # current
        R = self.R.T
        return dot(R, self.w) + self.td*z_q
    
    def getVelocity(self):  # Ri_i-1 (wxp + ^{i-1}v_{i-1}) + ddot 
        a = self.v + cross(self.w,self.P, axis=0)
        v1 = dot(self.R.T, a) #+ self.d_dot
        return v1
    
    def getVelocityOfCOM(self, world=True):
        w = self.getRotation_w()
        v = cross(w, self.P_c, axis=0) + self.getVelocity()  # current w plus v-1 + v
        if world is True :
            v_world = dot(self.ROT, v)
            return v_world
        else :
            return w + v
    
    def getKinectEnergy(self):
        d_dot = self.d_dot * z_q
        m = self.m
        w = self.getRotation_w()
        v = self.getVelocityOfCOM(world=True)
        I = self.getI()
        
        if d_dot is not 0:
            T2 = 1/2*m*dot(d_dot.T, d_dot)
        else:
            T2 = 0
        T = 1/2*m*dot(v.T, v)
        M = 1/2*dot(w.T, dot(I, w))
        return T + M + T2
    
    def getPotential(self):
        return self.m * self.Pot

In [350]:
# _P, _Pc, _m, _r, _t, 
    #_w, I_, _d, _v, _ROT):
link1 = jointHandler(P_01, P_1c, m_1, r_1, t_1, 
                     o_q, [I_xx1, I_yy1, I_zz1], 0, 0, R(t_1), 0)

link2 = jointHandler(P_12, P_2c, m_2, r_2, t_2, 
                     link1.getRotation_w(), [I_xx2, I_yy2, I_zz2], 0, 
                     link1.getVelocity(), dot(R(t_1), R(t_2)), link1.Pot)

# theta_3 = 0
link3 = jointHandler(P_23, P_3c, m_3, r_3, 0,    
                     link2.getRotation_w(), [I_xx3, I_yy3, I_zz3], d_3,
                     link2.getVelocity(), dot(dot(R(t_1), R(t_2)), R(0)), link2.Pot)

link4 = jointHandler(P_34, P_4c, m_4, r_4, t_4,
                     link3.getRotation_w(), [0, 0, 0], d_3,
                     link3.getVelocity(), dot(dot(dot(R(t_1), R(t_2)), R(0)), R(t_4)), 
                     (link3.Pot))

## Lagrange 

### First Parameter

In [351]:
M11t = diff(link1.getKinectEnergy(), diff(t_1, t))
M11tt = simplify(diff(M11t, t))

In [352]:
M22t = diff(link2.getKinectEnergy(), diff(t_2, t))
M22tt = simplify(diff(M22t, t))

M21t = diff(link2.getKinectEnergy(), diff(t_1, t))
M21tt = simplify(diff(M21t, t))

In [353]:
M33t = diff(link3.getKinectEnergy(), diff(d_3, t))
M33tt = simplify(diff(M33t, t))

M32t = diff(link3.getKinectEnergy(), diff(t_2, t))
M32tt = simplify(diff(M32t, t))

M31t = diff(link3.getKinectEnergy(), diff(t_1, t))
M31tt = simplify(diff(M31t, t))

In [354]:
M44t = diff(link4.getKinectEnergy(), diff(t_4, t))
M44tt = simplify(diff(M44t, t))

M44t = diff(link4.getKinectEnergy(), diff(d_3, t))
M43tt = simplify(diff(M43t, t))

M44t = diff(link4.getKinectEnergy(), diff(t_2, t))
M42tt = simplify(diff(M42t, t))

M44t = diff(link4.getKinectEnergy(), diff(t_1, t))
M41tt = simplify(diff(M41t, t))

### Second Paramter

In [355]:
X11t = diff(link1.getKinectEnergy(), t_1)

In [356]:
X22t = diff(link2.getKinectEnergy(), t_2)
X21t = diff(link2.getKinectEnergy(), t_1)

In [357]:
X33t = diff(link3.getKinectEnergy(), d_3)
X32t = diff(link3.getKinectEnergy(), t_2)
X31t = diff(link3.getKinectEnergy(), t_1)

In [358]:
X44t = diff(link4.getKinectEnergy(), t_4)
X43t = diff(link3.getKinectEnergy(), d_3)
X42t = diff(link2.getKinectEnergy(), t_2)
X41t = diff(link1.getKinectEnergy(), t_1)

### Potential diff

In [367]:
V = link1.getPotential() +  link2.getPotential() + \
                                 link3.getPotential() +link4.getPotential()
V1 = diff(V, t_1)*g
V2 = diff(V, t_2)*g
V3 = diff(V, d_3)*g
V4 = diff(V, t_4)*g

In [368]:
V = np.array([V1, V2, V3, V4]).reshape(4, -1)

In [369]:
V = np.array([V1, V2, V3, V4]).reshape(4, -1)
X = np.array([X11t+X21t+X31t+X41t, X22t+X32t+X42t, X33t+X43t, X44t]).reshape(4, -1)
M = np.array([M11tt+M21tt+M31tt+M41tt, M22tt+M32tt+M42tt, M33tt+M43tt, M44tt]).reshape(4, -1)

In [375]:
tau = M - X + V
latex(simplify(tau[0]))

'\\left[\\begin{matrix}I_{zz2} \\frac{d^{2}}{d t^{2}} \\theta_{1}{\\left (t \\right )} + I_{zz2} \\frac{d^{2}}{d t^{2}} \\theta_{2}{\\left (t \\right )} + I_{zz3} \\frac{d^{2}}{d t^{2}} \\theta_{1}{\\left (t \\right )} + I_{zz3} \\frac{d^{2}}{d t^{2}} \\theta_{2}{\\left (t \\right )} + a_{1}^{2} m_{2} \\frac{d^{2}}{d t^{2}} \\theta_{1}{\\left (t \\right )} + a_{1}^{2} m_{3} \\frac{d^{2}}{d t^{2}} \\theta_{1}{\\left (t \\right )} - 2 a_{1} a_{2} m_{3} \\sin{\\left (\\theta_{2}{\\left (t \\right )} \\right )} \\frac{d}{d t} \\theta_{1}{\\left (t \\right )} \\frac{d}{d t} \\theta_{2}{\\left (t \\right )} - a_{1} a_{2} m_{3} \\sin{\\left (\\theta_{2}{\\left (t \\right )} \\right )} \\left(\\frac{d}{d t} \\theta_{2}{\\left (t \\right )}\\right)^{2} + 2 a_{1} a_{2} m_{3} \\cos{\\left (\\theta_{2}{\\left (t \\right )} \\right )} \\frac{d^{2}}{d t^{2}} \\theta_{1}{\\left (t \\right )} + a_{1} a_{2} m_{3} \\cos{\\left (\\theta_{2}{\\left (t \\right )} \\right )} \\frac{d^{2}}{d t^{2}} \\theta_{

In [362]:
V = link1.getPotential() +  link2.getPotential() + \
                                 link3.getPotential() +link4.getPotential()
V

array([-d_1*m_1 + m_2*(-d_1 - d_2) + m_3*(-d_1 - d_2 + r_3 - d_3(t)) + m_4*(-d_1 - d_2 + r_3 - d_3(t))],
      dtype=object)

array([-d_1*m_1 + m_2*(-d_1 - d_2) + m_3*(-d_1 - d_2 + r_3 - theta_3(t)) + m_4*(-d_1 - d_2 + r_3 - theta_3(t))],
      dtype=object)